# YamboWorkflow: a full DFT+MBPT flow - BSE version

This is the core workflow of the plugin. It runs a DFT+GW/BSE study, involving automatic error handling and skipping the already done steps - if a parent calculation is provided.

The idea is to create a builder instance, which collects all the inputs needed for the simulation, then submit it by means of the aiida "submit" function.

In [61]:
from aiida import orm, load_profile
load_profile()

Profile<uuid='b35700dae723411ea16ebc82d58f16bc' name='mb'>

## 1 - Creation of YamboWorkflow instance using protocols

In this tutorial the creation of the builder, for a given workflow, is provided by means of the get_builder_from_protocol function.
This is, for now, just a way to automatically populated all the needed inputs, allowing the user to partially skip the long part of input definition as done in examples like the one contained in ``aiida_yambo/examples_hBN/workflows/yambo_workflow.py``.


It is not meant to be an already tested set of default values as instead is done for the aiida-quantumespresso plugin. 

In [62]:
from aiida.plugins import WorkflowFactory

In [63]:
YamboWorkflow = WorkflowFactory('yambo.yambo.yambowf')

## minimal inputs needed for protocols

In [64]:
options = {
    'pwcode_id': 'pw-7.1@hydralogin', 
    'pseudo_family':"PseudoDojo/0.4/PBE/SR/standard/upf",
    'yamboprecode_id':'p2y-5.1@hydralogin',
    'yambocode_id':'yambo-5.1@hydralogin',
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; take your previously nscf id (pk) to skip the DFT part.
    'structure_id':161,
}

In [65]:
#YamboWorkflow.get_builder_from_protocol??

In [66]:
from aiida_quantumespresso.common.types import ElectronicType

## Here we select BSE

In [67]:
builder = YamboWorkflow.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= orm.load_node(options['structure_id']),
            overrides={},
            pseudo_family= options['pseudo_family'],
            #parent_folder=orm.load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='bse', #or 'bse'; default is 'gw'
)


Summary of the main inputs:
BndsRnXs = 200
NGsBlkXs = 6 Ry
BSENGBlk = 6 Ry
FFTGvecs = 18 Ry


kpoint mesh for nscf: [6, 6, 2]


With respect to the previous examples (2_YamboRestart), we can see that we have a different FFTGvecs value and a kpoint mesh: the first is due to the fact that now FFTGvecs is estimated from aiida-quantumespresso protocol, not from the previous DFT run. The second is needed in order to have also the DFT part performed, if needed (no previous parent calculations).

In [68]:
#You can also try different protocols:
    
YamboWorkflow.get_available_protocols()

{'fast': {'description': 'Under converged for most materials, but fast'},
 'moderate': {'description': 'Meta converged for most materials, higher computational cost than fast'},
 'precise': {'description': 'Converged for most materials, higher computational cost than moderate'}}

Now, if you inspect the prepopulated inputs, you can see the default values respecting the imposed protocol:

In [69]:
builder.nscf.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'nscf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 60.0,
  'ecutrho': 480.0,
  'force_symmorphic': True,
  'nbnd': 200},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'conv_thr': 1.6e-09}}

In [70]:
builder.yres.yambo.parameters.get_dict()

{'arguments': ['em1s', 'bse', 'bss', 'optics', 'rim_cut', 'dipoles'],
 'variables': {'Chimod': 'hartree',
  'DysSolver': 'n',
  'GTermKind': 'BG',
  'X_and_IO_nCPU_LinAlg_INV': [1, ''],
  'BSEmod': 'resonant',
  'BSKmod': 'SEX',
  'BSSmod': 'd',
  'Lkind': 'full',
  'BSEQptR': [[1, 1], ''],
  'FFTGvecs': [18, 'Ry'],
  'GbndRnge': [[1, 200], ''],
  'BndsRnXs': [[1, 200], ''],
  'NGsBlkXs': [6, 'Ry'],
  'BSENGBlk': [6, 'Ry']}}

## 2 - Inputs completion

We have to include also the resources:

In [71]:
builder.scf.pw.metadata.options = {
    'max_wallclock_seconds': 60*60, # in seconds
    'resources': {
            "num_machines": 1, # nodes
            "num_mpiprocs_per_machine": 16, # MPI per nodes
            "num_cores_per_mpiproc": 1, # OPENMP
        },
    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
    #'account':'project_name',
    'queue_name':'s3par',
    #'qos':'',
}

builder.nscf.pw.metadata.options = builder.scf.pw.metadata.options
builder.yres.yambo.metadata.options = builder.scf.pw.metadata.options

## 3 - Overrides

It is possible to modify the default inputs also during the builder creation phase, so not a posteriori. This can be done by using overrides:

In [72]:
overrides_scf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw':{
            
        'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
        },
        },
    }

overrides_nscf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw': {
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{},
                'ELECTRONS':{'diagonalization':'cg'},
            },
             'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
        },
    },
}

overrides_yambo = {
        "yambo": {
            "parameters": {
                "arguments": [
                    "rim_cut",
                ],
                "variables": {
                    "NGsBlkXs": [4, "Ry"],
                    "BSENGBlk": [4, "Ry"],
                    "FFTGvecs": [20, "Ry"],
                },
            },
        'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed, i.e. in PBS/Torque 
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
                    },
        },
    
}

overrides = {
    'yres': overrides_yambo,
    'nscf': overrides_nscf,
    'scf': overrides_scf
    
}


In [73]:
builder = YamboWorkflow.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= orm.load_node(options['structure_id']),
            overrides=overrides,
            #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='bse', #or 'bse'; default is 'gw'
)

Summary of the main inputs:
BndsRnXs = 200
NGsBlkXs = 4 Ry
BSENGBlk = 4 Ry
FFTGvecs = 20 Ry


kpoint mesh for nscf: [6, 6, 2]


In [74]:
builder.yres.yambo.parameters = orm.Dict(
    dict={'arguments':['em1s','bse','bss','optics', 'dipoles',],
                'variables':{
                'BSEmod': 'resonant',
                'BSKmod': 'SEX',
                'BSSmod': 'd',
                'Lkind': 'full',
                'NGsBlkXs': [2, 'Ry'],
                'BSENGBlk': [2, 'Ry'],
                'Chimod': 'hartree',
                'DysSolver': 'n',
                'BEnSteps': [10,''],
                'BSEQptR': [[1,1],''],   # <=== if you do not set your Q-vector, it will detects the one of the minimum gap. 
                'BSEBands': [[8,9],''],
                'BEnRange': [[0.0, 10.0],'eV'],
                'BDmRange': [[0.1, 0.1],'eV'],
                'BLongDir': [[1.0, 1.0, 1.0],''],
                'LongDrXp': [[1.0, 1.0, 1.0],''],
                'LongDrXd': [[1.0, 1.0, 1.0],''],
                'LongDrXs': [[1.0, 1.0, 1.0],''],
                'BndsRnXs': [[1,50], ''],
                #'KfnQP_E':[[1.5,1,1],''],
                # 'KfnQPdb': 'E < ./ndb.QP', <== it is automatically set in the workflow.
                'BS_CPU':str(int(16/2))+' 2 1',
                'BS_ROLEs':'k eh t',
                },}
)


### Providing the inputs for the G0W0 quasiparticles simulation, pro BSE.

In the following we provide the instructions on the quasiparticles to be computed at the G0W0, and the parameters for the simulation. 
The parameters for the G0W0 run are stored in under `builder.qp` (NB: `builder.yres` is used for the final BSE parameters)

In [75]:
QP_subset_dict= {
    'range_QP':6, #eV         , default=nscf_gap_eV*1.2
    'range_spectrum':10, #eV

}

QP_subset_dict.update({
    'split_bands':True, #default
    'extend_QP': True, #default is False
    'consider_only':[8,9], #we explicitely compute only these bands.
    'T_smearing':1e-2, #default
    'qp_per_subset': 20,
    'parallel_runs':4,
})

builder.QP_subset_dict= orm.Dict(dict=QP_subset_dict)
builder.qp = builder.yres #we provide the same inputs for G0W0 and BSE, namely resources and settings. 

#providing the G0W0 input parameters.
params_gw = {
    'arguments': [
        'dipoles',
        'HF_and_locXC',
        'dipoles',
        'gw0',
        'ppa',],
    'variables': {
        'Chimod': 'hartree',
        'DysSolver': 'n',
        'GTermKind': 'BG',
        'NGsBlkXp': [2, 'Ry'],
        'BndsRnXp': [[1, 50], ''],
        'GbndRnge': [[1, 50], ''],
        'QPkrange': [[[1, 1, 8, 9]], ''],}}


params_gw = orm.Dict(dict=params_gw)
builder.qp.yambo.parameters = params_gw

In [76]:
builder.yres.yambo.metadata.options

{'stash': {}, 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 16, 'num_cores_per_mpiproc': 1}, 'max_wallclock_seconds': 3600, 'withmpi': True, 'prepend_text': 'export OMP_NUM_THREADS=1', 'queue_name': 's3par'}

In [77]:
builder.nscf.pw.parameters.get_dict()['ELECTRONS']['diagonalization']

'cg'

In [78]:
family = orm.load_group("PseudoDojo/0.4/PBE/SR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.scf.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 
builder.nscf.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 

In [79]:
builder.additional_parsing = orm.List(list=['gap_','G_v','gap_GG','gap_GY','gap_GK','gap_KK','gap_GM','lowest_exciton','brightest_exciton'])

## 5 RUN

In [80]:
from aiida.engine import submit

In [81]:
run = None

In [82]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


uuid: a8a1d906-4644-4140-a145-08a6e9c8ebaf (pk: 6051) (aiida.workflows:yambo.yambo.yambowf)


# Inspecting the outputs

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance. All the outputs of YamboRestart and YamboCalculation are inherited

In [84]:
run.is_finished_ok

True

In [85]:
run.outputs.output_ywfl_parameters.get_dict()

{'QP': 6150,
 'SOC': False,
 'gap_': 5.7658574014878,
 'homo': -0.14813481971379,
 'lumo': 5.617722581774,
 'c_max': 9,
 'q_ind': 12,
 'v_min': 8,
 'gap_GG': 7.6526870309934,
 'gap_GK': 8.4105505276009,
 'gap_GM': 7.3718826064184,
 'gap_GW': 5.7659,
 'gap_KK': 6.9361484812595,
 'homo_G': -1.7541600246444,
 'homo_K': -0.27975797830299,
 'lumo_G': 5.898527006349,
 'lumo_K': 6.6563905029565,
 'lumo_M': 5.617722581774,
 'gap_DFT': 4.2863,
 'gap_dft': 4.286313267383,
 'nscf_pk': 6094,
 'homo_dft': 0.0,
 'lumo_dft': 4.286313267383,
 'gap_GG_dft': 6.7830028990895,
 'gap_GK_dft': 6.249396599409,
 'gap_GM_dft': 5.5682161341578,
 'gap_KK_dft': 5.0788206942558,
 'homo_G_dft': -1.2819028667748,
 'homo_K_dft': -0.11132696162164,
 'lumo_G_dft': 5.5011000323147,
 'lumo_K_dft': 4.9674937326342,
 'lumo_M_dft': 4.286313267383,
 'lowest_exciton': 5.5415873527527,
 'brightest_exciton': 10.772668838501,
 'candidate_for_BSE': True,
 'lowest_exciton_index': 1,
 'brightest_exciton_index': 60}

#### Description of all the outputs... for example gap GW....



In [86]:
run.outputs.nscf_mapping.get_dict()

{'soc': False,
 'gap_': [[14, 14, 8, 8], [7, 7, 9, 9]],
 'gap_GG': [[1, 1, 8, 8], [1, 1, 9, 9]],
 'gap_GK': [[1, 1, 8, 8], [13, 13, 9, 9]],
 'gap_GM': [[1, 1, 8, 8], [7, 7, 9, 9]],
 'gap_KK': [[13, 13, 8, 8], [13, 13, 9, 9]],
 'homo_k': 14,
 'lumo_k': 7,
 'valence': 8,
 'gap_type': 'indirect',
 'conduction': 9,
 'nscf_gap_eV': 4.286,
 'dft_predicted': 'semiconductor/insulator',
 'number_of_kpoints': 14,
 'magnetic_calculation': False}